In [34]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

# -------------------
# SETTINGS
# -------------------
DB_FILE = "boxing_odds_staging.db"
TABLE_NAME = "odds_history"
OUTPUT_FILE = "best_odds_interactive.html"

# Optional bookmaker logos
bookmaker_logos = {
    "Virgin Bet": "https://upload.wikimedia.org/path-to-virginbet-logo.png",
    "1xBet": "https://upload.wikimedia.org/path-to-1xbet-logo.png"
}

# -------------------
# DATABASE QUERY
# -------------------
conn = sqlite3.connect(DB_FILE)

# Latest odds per fighter/bookmaker/event
latest_query = f"""
SELECT *
FROM {TABLE_NAME}
WHERE (event_id, fighter, bookmaker, observed_at) IN (
    SELECT event_id, fighter, bookmaker, MAX(observed_at)
    FROM {TABLE_NAME}
    GROUP BY event_id, fighter, bookmaker
)
"""
latest_df = pd.read_sql(latest_query, conn)

# Historical odds for plotting
history_query = f"""
SELECT event_id, fighter, decimal_odds, observed_at, bookmaker
FROM {TABLE_NAME}
"""
history_df = pd.read_sql(history_query, conn)
conn.close()

# Format date
latest_df["Fight Date"] = pd.to_datetime(latest_df["commence_time"]).dt.date
latest_df["Matchup"] = latest_df["home_team"] + " vs " + latest_df["away_team"]

# Get best odds per fighter
best_odds_df = latest_df.loc[latest_df.groupby(["event_id", "fighter"])["decimal_odds"].idxmax()]
best_odds_df = best_odds_df[["Fight Date", "Matchup", "fighter", "bookmaker", "decimal_odds", "event_id"]]
best_odds_df.columns = ["Fight Date", "Matchup", "Fighter", "Best Bookmaker", "Best Odds", "event_id"]

# -------------------
# BUILD HTML
# -------------------
html_parts = ["""
<html>
<head>
<title>Best Odds with Charts</title>
<style>
    body { font-family: Arial, sans-serif; background: #f5f7fa; margin: 0; padding: 20px; }
    h1 { text-align: center; margin-bottom: 30px; }
    h2 { margin-top: 40px; color: #333; }
    table { border-collapse: collapse; width: 100%; background: white; border-radius: 8px; overflow: hidden; box-shadow: 0 2px 8px rgba(0,0,0,0.05); }
    th, td { padding: 12px 15px; text-align: left; }
    th { background-color: #f0f2f5; font-weight: bold; }
    tr:nth-child(even) { background-color: #fafafa; }
    tr.clickable:hover { background-color: #e8f4ff; cursor: pointer; }
    .chart-container { display: none; }
    .chart-cell { background: #fff; padding: 20px; box-shadow: inset 0 0 5px rgba(0,0,0,0.05); }
    .toggle-icon { float: right; font-size: 14px; color: #666; }
    .fade-in { animation: fadeIn 0.3s ease-in-out; }
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(-5px); }
        to { opacity: 1; transform: translateY(0); }
    }
</style>
<script>
    function toggleChart(id, iconId) {
        var row = document.getElementById(id);
        var icon = document.getElementById(iconId);
        if (row.style.display === 'table-row') {
            row.style.display = 'none';
            icon.innerHTML = '▼';
        } else {
            row.style.display = 'table-row';
            row.classList.add('fade-in');
            icon.innerHTML = '▲';
        }
    }
</script>
</head>
<body>
<h1>Best Odds (Interactive)</h1>
"""]

# Group fights by date
for fight_date, group in best_odds_df.groupby("Fight Date"):
    html_parts.append(f"<h2>{fight_date}</h2>")
    html_parts.append("<table>")
    html_parts.append("<tr><th>Matchup</th><th>Fighter</th><th>Best Bookmaker</th><th>Best Odds</th></tr>")
    
    for _, row in group.iterrows():
        fid = f"chart_{row['event_id']}_{row['Fighter']}".replace(" ", "_").replace("'", "")
        icon_id = f"icon_{row['event_id']}_{row['Fighter']}".replace(" ", "_").replace("'", "")

        # Best bookmaker cell (logo if exists)
        bookmaker_cell = row["Best Bookmaker"]
        if row["Best Bookmaker"] in bookmaker_logos:
            bookmaker_cell = f"<img src='{bookmaker_logos[row['Best Bookmaker']]}' height='20'> {row['Best Bookmaker']}"

        # Clickable fighter row
        html_parts.append(
            f"<tr class='clickable' onclick=\"toggleChart('{fid}', '{icon_id}')\">"
            f"<td>{row['Matchup']}</td>"
            f"<td>{row['Fighter']} <span class='toggle-icon' id='{icon_id}'>▼</span></td>"
            f"<td>{bookmaker_cell}</td>"
            f"<td>{row['Best Odds']}</td></tr>"
        )

        # Chart data for fighter
        fighter_hist = history_df[
            (history_df["event_id"] == row["event_id"]) & 
            (history_df["fighter"] == row["Fighter"])
        ].copy()
        fighter_hist["observed_at"] = pd.to_datetime(fighter_hist["observed_at"])

        fig = go.Figure()
        for book in fighter_hist["bookmaker"].unique():
            sub_df = fighter_hist[fighter_hist["bookmaker"] == book]
            fig.add_trace(go.Scatter(
                x=sub_df["observed_at"], y=sub_df["decimal_odds"], mode="lines+markers", name=book
            ))

        fig.update_layout(
            title=f"Odds Movement - {row['Fighter']}",
            xaxis_title="Date",
            yaxis_title="Decimal Odds",
            template="plotly_white",
            height=400
        )

        chart_html = fig.to_html(full_html=False, include_plotlyjs="cdn")

        # Hidden chart row
        html_parts.append(f"<tr id='{fid}' class='chart-container'><td colspan='4' class='chart-cell'>{chart_html}</td></tr>")

    html_parts.append("</table>")

html_parts.append("</body></html>")

# -------------------
# SAVE HTML
# -------------------
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write("".join(html_parts))

print(f"✅ Interactive HTML saved to {OUTPUT_FILE}")


✅ Interactive HTML saved to best_odds_interactive.html
